# Data Cleaning Listado de Listado de Beneficiarios 2023

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. Beneficiarios2023 -> Dataset con el listado de TODOS los beneficiarios.
2. Estados_Beneficiarios 2023 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='cp1252', index_col=0, skiprows=1) for file in csv_files]

    # Print the number of rows for each DataFrame
    for i, df in enumerate(dataframes):
        print(f"Number of rows in DataFrame {i+1}: {df.shape[0]}")

    # Calculate the sum of rows in each individual dataset
    individual_row_sum = sum([df.shape[0] for df in dataframes])

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df, individual_row_sum


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

### Lectura del dataset del INEGI

In [3]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252', dtype={'CVE_ENT': str, 'CVE_MUN': str})

### Lectura del listado de Beneficiarios 2023

In [4]:
listado_beneficiarios, sumOfRows = load_datasets("../../data/productores_beneficiarios")

Number of rows in DataFrame 1: 42853
Number of rows in DataFrame 2: 493512
Number of rows in DataFrame 3: 830588
Number of rows in DataFrame 4: 312842


In [5]:
nan_rows = listado_beneficiarios.isna()

In [6]:
sumOfRows

1679795

In [7]:
listado_beneficiarios.head()

,ESTADO,MUNICIPIO,ACUSE ESTATAL,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,SINALOA,AHOME,23-PROESFE-ESTR-000051-E000-SL,ABOITES,ARMENTA,FRANCISCO,4.0
1,SINALOA,AHOME,23-PROESFE-ESTR-043733-E000-SL,ABOYTE,RUIZ,ISMAEL,7.0
2,SINALOA,AHOME,23-PROESFE-ESTR-000052-E000-SL,ABOYTES,ARMENTA,RODOLFO,4.0
3,SINALOA,AHOME,23-PROESFE-ESTR-030724-E000-SL,ACOSTA,BUELNA,EDGAR,10.0
4,SINALOA,AHOME,23-PROESFE-ESTR-035928-E000-SL,ACOSTA,BUELNA,GUADALUPE,5.0


# 2. Limpieza de los datos

### 2.1 INEGI

In [8]:
# Revisamos las columnas del dataset
dataset_inegi.columns

Index(['MAPA', 'Estatus', 'CVE_ENT', 'NOM_ENT', 'NOM_ABR', 'CVE_MUN',
       'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
       'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
       'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS'],
      dtype='object')

In [9]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [10]:
# Renombrar las columnas
dataset_inegi.rename(columns={
    'CVE_ENT': 'CVE_ENT',
    'NOM_ENT': 'Entidad_inegi',
    'CVE_MUN': 'CVE_MUN',
    'NOM_MUN': 'Municipio_inegi'
}, inplace=True)


# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

CVE_ENT            object
Entidad_inegi      object
CVE_MUN            object
Municipio_inegi    object
dtype: object

In [11]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_uniquemun = dataset_inegi.drop_duplicates()
print("Shape of dataset_inegi_uniquemun: ", dataset_inegi_uniquemun.shape)

Shape of dataset_inegi:  (299568, 4)
Shape of dataset_inegi_uniquemun:  (2476, 4)


A partir de aquí seguimos trabajando con el listado de Estados y Municipios limpio de Inegi (sin repetir) "dataset_inegi_clean".

In [12]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_uniquemun.head()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
0,01,Aguascalientes,001,Aguascalientes
708,01,Aguascalientes,002,Asientos
945,01,Aguascalientes,003,Calvillo
1237,01,Aguascalientes,004,Cosío
1330,01,Aguascalientes,005,Jesús María


In [13]:
dataset_inegi_uniquemun.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2476 entries, 0 to 299526
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CVE_ENT          2476 non-null   object
 1   Entidad_inegi    2476 non-null   object
 2   CVE_MUN          2476 non-null   object
 3   Municipio_inegi  2476 non-null   object
dtypes: object(4)
memory usage: 96.7+ KB


In [14]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_uniquemun.nunique())

Los valores únicos en cada columna son:
 CVE_ENT              32
Entidad_inegi        32
CVE_MUN             570
Municipio_inegi    2332
dtype: int64


#### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [15]:
# Estandarizamos la limpieza de los datos
dataset_inegi_uniquemun['Entidad_c_inegi'] = dataset_inegi_uniquemun['Entidad_inegi'].apply(clean_text)
dataset_inegi_uniquemun['Municipio_c_inegi'] = dataset_inegi_uniquemun['Municipio_inegi'].apply(clean_text)

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_63913/4159994235.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['Entidad_c_inegi'] = dataset_inegi_uniquemun['Entidad_inegi'].apply(clean_text)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_63913/4159994235.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['Municipio_c_inegi'] = dataset_inegi_uniquemun['Municipio_inegi'].apply(clean_text)


In [16]:
# Creamos una columna con la clave única por municipio

dataset_inegi_uniquemun['KEY_inegi'] = dataset_inegi_uniquemun['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniquemun[
    'Municipio_c_inegi'].astype(str)

dataset_inegi_uniquemun.head()

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_63913/2100087808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_uniquemun['KEY_inegi'] = dataset_inegi_uniquemun['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_uniquemun[


,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi,KEY_inegi
0,01,Aguascalientes,001,Aguascalientes,aguascalientes,aguascalientes,aguascalientes-aguascalientes
708,01,Aguascalientes,002,Asientos,aguascalientes,asientos,aguascalientes-asientos
945,01,Aguascalientes,003,Calvillo,aguascalientes,calvillo,aguascalientes-calvillo
1237,01,Aguascalientes,004,Cosío,aguascalientes,cosio,aguascalientes-cosio
1330,01,Aguascalientes,005,Jesús María,aguascalientes,jesus maria,aguascalientes-jesus maria


In [17]:
dataset_inegi_uniquemun.tail()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi,KEY_inegi
299150,32,Zacatecas,054,Villa Hidalgo,zacatecas,villa hidalgo,zacatecas-villa hidalgo
299211,32,Zacatecas,055,Villanueva,zacatecas,villanueva,zacatecas-villanueva
299363,32,Zacatecas,056,Zacatecas,zacatecas,zacatecas,zacatecas-zacatecas
299484,32,Zacatecas,057,Trancoso,zacatecas,trancoso,zacatecas-trancoso
299526,32,Zacatecas,058,Santa María de la Paz,zacatecas,santa maria de la paz,zacatecas-santa maria de la paz


## 2.2 Beneficiarios 2023

### Creación de Estados_Beneficiarios2023
Este dataset es una versión de Beneficiarios2023 pero más ligera y sin repeticiones.

In [18]:
listado_beneficiarios.dtypes

ESTADO               object
MUNICIPIO            object
ACUSE ESTATAL        object
APELLIDO PATERNO     object
APELLIDO MATERNO     object
NOMBRE (S)           object
PAQUETE             float64
dtype: object

In [19]:
descriptive_stats = listado_beneficiarios.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                      count   unique                             top    freq  \
ESTADO              1679795       30                        GUERRERO  331011   
MUNICIPIO           1679795     2237                  LAS MARGARITAS   18040   
ACUSE ESTATAL       1679795  1679795  23-PROESFE-ESTR-000051-E000-SL       1   
APELLIDO PATERNO    1679794    14994                       HERNANDEZ   77403   
APELLIDO MATERNO    1653572    17599                       HERNANDEZ   76938   
NOMBRE (S)          1679795   124611                            JUAN   26791   
PAQUETE           1679794.0      NaN                             NaN     NaN   

                     mean       std  min  25%  50%  75%   max  
ESTADO                NaN       NaN  NaN  NaN  NaN  NaN   NaN  
MUNICIPIO             NaN       NaN  NaN  NaN  NaN  NaN   NaN  
ACUSE ESTATAL         NaN       NaN  NaN  NaN  NaN  NaN   NaN  
APELLIDO PATERNO      NaN       NaN  NaN  NaN  NaN  NaN   NaN  
APELLIDO MATERNO      NaN       NaN  Na

In [20]:
listado_beneficiarios.isna().sum()

ESTADO                  0
MUNICIPIO               0
ACUSE ESTATAL           0
APELLIDO PATERNO        1
APELLIDO MATERNO    26223
NOMBRE (S)              0
PAQUETE                 1
dtype: int64

In [21]:
listado_beneficiarios.head()

,ESTADO,MUNICIPIO,ACUSE ESTATAL,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,SINALOA,AHOME,23-PROESFE-ESTR-000051-E000-SL,ABOITES,ARMENTA,FRANCISCO,4.0
1,SINALOA,AHOME,23-PROESFE-ESTR-043733-E000-SL,ABOYTE,RUIZ,ISMAEL,7.0
2,SINALOA,AHOME,23-PROESFE-ESTR-000052-E000-SL,ABOYTES,ARMENTA,RODOLFO,4.0
3,SINALOA,AHOME,23-PROESFE-ESTR-030724-E000-SL,ACOSTA,BUELNA,EDGAR,10.0
4,SINALOA,AHOME,23-PROESFE-ESTR-035928-E000-SL,ACOSTA,BUELNA,GUADALUPE,5.0


In [22]:
Estados_beneficiarios2023 = listado_beneficiarios[['ESTADO', 'MUNICIPIO']]

In [23]:
Estados_beneficiarios2023 = Estados_beneficiarios2023.drop_duplicates()

In [24]:
# Estandarizamos la limpieza de los datos
Estados_beneficiarios2023['ESTADO_c_benef'] = Estados_beneficiarios2023['ESTADO'].apply(clean_text)
Estados_beneficiarios2023['MUNICIPIO_c_benef'] = Estados_beneficiarios2023['MUNICIPIO'].apply(clean_text)

In [25]:
Estados_beneficiarios2023.shape

(2358, 4)

In [26]:
Estados_beneficiarios2023['KEY_benef'] = Estados_beneficiarios2023['ESTADO_c_benef'].astype(str) + '-' + Estados_beneficiarios2023[
    'MUNICIPIO_c_benef'].astype(str)

Estados_beneficiarios2023.head()


,ESTADO,MUNICIPIO,ESTADO_c_benef,MUNICIPIO_c_benef,KEY_benef
0,SINALOA,AHOME,sinaloa,ahome,sinaloa-ahome
2574,SINALOA,ANGOSTURA,sinaloa,angostura,sinaloa-angostura
3789,SINALOA,BADIRAGUATO,sinaloa,badiraguato,sinaloa-badiraguato
6623,SINALOA,CHOIX,sinaloa,choix,sinaloa-choix
9113,SINALOA,CONCORDIA,sinaloa,concordia,sinaloa-concordia


In [27]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = Estados_beneficiarios2023.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                  count unique                        top freq
ESTADO             2358     30                     OAXACA  566
MUNICIPIO          2358   2237            EMILIANO ZAPATA    6
ESTADO_c_benef     2358     30                     oaxaca  566
MUNICIPIO_c_benef  2358   2235            emiliano zapata    6
KEY_benef          2358   2356  oaxaca-san pedro mixtepec    2


In [28]:
Estados_beneficiarios2023 = Estados_beneficiarios2023.drop_duplicates(subset='KEY_benef')

In [29]:
Estados_beneficiarios2023.shape

(2356, 5)

# 3. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2023

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Listado_beneficiarios2023. Para ello haremos un Left join entre Estados_Beneficiarios2023 y dataset_inegi_clean.

### 3.1 Creamos las columnas clave

Confirmamos que tengamos las columnas clave en abos datasets.

In [30]:
dataset_inegi_uniquemun.head()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi,KEY_inegi
0,01,Aguascalientes,001,Aguascalientes,aguascalientes,aguascalientes,aguascalientes-aguascalientes
708,01,Aguascalientes,002,Asientos,aguascalientes,asientos,aguascalientes-asientos
945,01,Aguascalientes,003,Calvillo,aguascalientes,calvillo,aguascalientes-calvillo
1237,01,Aguascalientes,004,Cosío,aguascalientes,cosio,aguascalientes-cosio
1330,01,Aguascalientes,005,Jesús María,aguascalientes,jesus maria,aguascalientes-jesus maria


In [31]:
Estados_beneficiarios2023.head()

,ESTADO,MUNICIPIO,ESTADO_c_benef,MUNICIPIO_c_benef,KEY_benef
0,SINALOA,AHOME,sinaloa,ahome,sinaloa-ahome
2574,SINALOA,ANGOSTURA,sinaloa,angostura,sinaloa-angostura
3789,SINALOA,BADIRAGUATO,sinaloa,badiraguato,sinaloa-badiraguato
6623,SINALOA,CHOIX,sinaloa,choix,sinaloa-choix
9113,SINALOA,CONCORDIA,sinaloa,concordia,sinaloa-concordia


In [32]:
dataset_inegi_uniquemun.shape

(2476, 7)

In [33]:
Estados_beneficiarios2023.shape

(2356, 5)

### 3.1 Left join (Para crear diccionario)

Creamos el diccionario.

In [34]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 90 y 100 de coincidencia
def fuzzy_merge_benef2023(df_benef, df_inegi, key1, key2, threshold=90, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [35]:
diccionario = fuzzy_merge_benef2023(Estados_beneficiarios2023, dataset_inegi_uniquemun, 'KEY_benef', 'KEY_inegi')
# Mostrar el resultado
diccionario.columns

Index(['ESTADO', 'MUNICIPIO', 'ESTADO_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef', 'best_match', 'match_score', 'CVE_ENT', 'Entidad_inegi',
       'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi',
       'KEY_inegi'],
      dtype='object')

In [36]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = diccionario.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                    count unique                               top freq  \
ESTADO               2358     30                            OAXACA  566   
MUNICIPIO            2358   2235                   EMILIANO ZAPATA    6   
ESTADO_c_benef       2358     30                            oaxaca  566   
MUNICIPIO_c_benef    2358   2235                   emiliano zapata    6   
KEY_benef            2358   2356         oaxaca-san pedro mixtepec    2   
best_match           2340   2315  chiapas-rincon chamula san pedro    3   
match_score        2340.0    NaN                               NaN  NaN   
CVE_ENT              2340     30                                20  563   
Entidad_inegi        2340     30                            Oaxaca  563   
CVE_MUN              2340    561                               003   29   
Municipio_inegi      2340   2197                            Ocampo    7   
Entidad_c_inegi      2340     30                            oaxaca  563   
Municipio_c_inegi    2340

In [37]:
nan_rows = diccionario.isna()

In [39]:
diccionario.to_csv('../../data/productores_beneficiarios/diccionarios_E2/diccionario_benef_E2.csv', index=False)

## 3.2 Listado beneficiarios2023

Esta sección se encarga de completar el listado original de Beneficiarios Autorizados con los nombre corregido de INEGI usando el diccionario.

In [40]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
listado_beneficiarios['ESTADO_Clean'] = listado_beneficiarios['ESTADO'].apply(clean_text)
listado_beneficiarios['MUNICIPIO_Clean'] = listado_beneficiarios['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
listado_beneficiarios['Estado-mun-KEY'] = listado_beneficiarios['ESTADO_Clean'].astype(str) + '-' + listado_beneficiarios[
    'MUNICIPIO_Clean'].astype(str)

In [41]:
nan_rows = listado_beneficiarios.isna()

In [42]:
nan_rows.sum()

ESTADO                  0
MUNICIPIO               0
ACUSE ESTATAL           0
APELLIDO PATERNO        1
APELLIDO MATERNO    26223
NOMBRE (S)              0
PAQUETE                 1
ESTADO_Clean            0
MUNICIPIO_Clean         0
Estado-mun-KEY          0
dtype: int64

In [43]:
print(listado_beneficiarios.shape)
listado_beneficiarios.columns


(1679795, 10)


Index(['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO',
       'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE', 'ESTADO_Clean',
       'MUNICIPIO_Clean', 'Estado-mun-KEY'],
      dtype='object')

In [ ]:
# Lectura del diccionario Verificado Simple
#diccionario_verificado = pd.read_csv('../../data/Diccionario_benef_manual_verificado.csv')
diccionario_verificado_simple = pd.read_csv('../../data/Diccionario_Simple.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../../data/Diccionario_Simple.csv'

In [ ]:
descriptive_stats = diccionario_verificado_simple.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                       count unique                               top freq  \
Unnamed: 0            2356.0    NaN                               NaN  NaN   
ï»¿KEY_benef            2356   2356                    mexico-acambay    1   
KEY_benef_Verificado    2356   2335  chiapas-rincon chamula san pedro    3   

                        mean         std  min     25%     50%      75%     max  
Unnamed: 0            1177.5  680.262939  0.0  588.75  1177.5  1766.25  2355.0  
ï»¿KEY_benef             NaN         NaN  NaN     NaN     NaN      NaN     NaN  
KEY_benef_Verificado     NaN         NaN  NaN     NaN     NaN      NaN     NaN  


In [ ]:
diccionario_verificado_simple.columns

Index(['Unnamed: 0', 'ï»¿KEY_benef', 'KEY_benef_Verificado'], dtype='object')

In [ ]:
diccionario_verificado_simple.rename(columns={'ï»¿KEY_benef': 'KEY_benef'}, inplace=True)

El diccionatio contiene KEY_benef únicos (Freq = 1), mientras que KEY_benef_Verificado puede repetirse.

### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos Listado_beneficiarios con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [ ]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(listado_beneficiarios, diccionario_verificado_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef", how='left', suffixes=('_benef', '_inegi'))

In [ ]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, dataset_inegi_uniquemun, left_on="KEY_benef_Verificado",
                                        right_on="KEY_inegi", how='left', suffixes=('_benef', '_inegi'))

In [ ]:
listado_beneficiarios_parte_II.columns

Index(['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO',
       'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE', 'ESTADO_Clean',
       'MUNICIPIO_Clean', 'Estado-mun-KEY', 'Unnamed: 0', 'KEY_benef',
       'KEY_benef_Verificado', 'CVE_ENT', 'Entidad_inegi', 'CVE_MUN',
       'Municipio_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi'],
      dtype='object')

In [ ]:
listado_beneficiarios_parte_II.shape

(1682494, 20)

In [ ]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = listado_beneficiarios_parte_II.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                          count   unique                             top  \
ESTADO                  1682494       30                        GUERRERO   
MUNICIPIO               1682494     2237                  LAS MARGARITAS   
ACUSE ESTATAL           1682494  1679795  23-PRONAFE-FERT-094967-S000-OC   
APELLIDO PATERNO        1682493    14994                       HERNANDEZ   
APELLIDO MATERNO        1656086    17599                       HERNANDEZ   
NOMBRE (S)              1682494   124611                            JUAN   
PAQUETE               1682493.0      NaN                             NaN   
ESTADO_Clean            1682494       30                        guerrero   
MUNICIPIO_Clean         1682494     2235                  las margaritas   
Estado-mun-KEY          1682494     2356          chiapas-las margaritas   
Unnamed: 0            1682494.0      NaN                             NaN   
KEY_benef               1682494     2356          chiapas-las margaritas   
KEY_benef_Ve

In [ ]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop_duplicates(subset = ['ACUSE ESTATAL'],keep = 'first')
listado_beneficiarios_parte_II.shape

(1679795, 20)

In [ ]:
# Seleccionamos las columnas que nos interesan
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II[
    ['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
     'NOMBRE (S)', 'PAQUETE', 'KEY_benef_Verificado', 'Entidad_inegi', 'Municipio_inegi', 'CVE_ENT', 'CVE_MUN']]

In [ ]:
# Revisamos el dataset
print(listado_beneficiarios_parte_II.shape)
print(listado_beneficiarios_parte_II.columns)
print(listado_beneficiarios_parte_II.head())

(1679795, 12)
Index(['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO',
       'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE', 'KEY_benef_Verificado',
       'Entidad_inegi', 'Municipio_inegi', 'CVE_ENT', 'CVE_MUN'],
      dtype='object')
    ESTADO MUNICIPIO                   ACUSE ESTATAL APELLIDO PATERNO  \
0  SINALOA     AHOME  23-PROESFE-ESTR-000051-E000-SL          ABOITES   
1  SINALOA     AHOME  23-PROESFE-ESTR-043733-E000-SL           ABOYTE   
2  SINALOA     AHOME  23-PROESFE-ESTR-000052-E000-SL          ABOYTES   
3  SINALOA     AHOME  23-PROESFE-ESTR-030724-E000-SL           ACOSTA   
4  SINALOA     AHOME  23-PROESFE-ESTR-035928-E000-SL           ACOSTA   

  APELLIDO MATERNO NOMBRE (S)  PAQUETE KEY_benef_Verificado Entidad_inegi  \
0          ARMENTA  FRANCISCO      4.0        sinaloa-ahome       Sinaloa   
1             RUIZ     ISMAEL      7.0        sinaloa-ahome       Sinaloa   
2          ARMENTA    RODOLFO      4.0        sinaloa-ahome       Sinaloa   
3       

In [ ]:
nan_counts = listado_beneficiarios_parte_II.isna().sum()
print(nan_counts)

ESTADO                      0
MUNICIPIO                   0
ACUSE ESTATAL               0
APELLIDO PATERNO            1
APELLIDO MATERNO        26223
NOMBRE (S)                  0
PAQUETE                     1
KEY_benef_Verificado        0
Entidad_inegi               0
Municipio_inegi             0
CVE_ENT                     0
CVE_MUN                     0
dtype: int64


Hay 0 filas que no han sido emparejadas.

In [ ]:
listado_beneficiarios_parte_II.to_csv('../../data/LISTADO_BENEFICIARIOS2023_COMPLETO.csv', index=False)

In [ ]:
listado_beneficiarios_parte_II.shape


(1679795, 12)